In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
import numpy as np
import torch.nn as nn
import torch.optim as optim
import cv2
import glob
from torchvision import models
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.tensorboard import SummaryWriter
from torch.utils.tensorboard import summary as summary_lib
from PIL import Image  # Add this import
import torchvision.models as models

In [2]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
transform = transforms.Compose([transforms.Resize((227, 227)), transforms.ToTensor()])

In [4]:
train_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)

Files already downloaded and verified


In [5]:
min_sigma = 0
max_sigma = 50/255

In [6]:
def add_noise(image, sigma):
    noise = torch.randn_like(image) * sigma
    noisy_image = torch.clamp(image + noise, 0, 1)
    return noisy_image

In [7]:
def save_noisy_image(img, name):
    img = img.view(img.size(0), 3, 227, 227)
    torchvision.utils.save_image(img, name)

In [8]:

from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, image_folder, sigma_values, transform=None, device=torch.device("cpu")):
        self.image_paths = glob.glob(os.path.join(image_folder, "*.png"))  # Adjust this for your image format
        self.sigma_values = sigma_values
        self.transform = transform
        self.device = device

    def __len__(self):
        return len(self.sigma_values)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)  # Convert NumPy array to PIL Image
        if self.transform:
            image = self.transform(image)
        sigma = torch.tensor(self.sigma_values[index], dtype=torch.float32, device=self.device)  # Ensure sigma is float32
        return image.to(self.device), sigma

In [9]:
def customFunction():
    sigma_values = []
    i = 1
    j = 1
    for inputs, _ in train_loader:
        sigma = np.random.uniform(min_sigma, max_sigma)
        sigma_values.append(sigma)
        noisy_inputs = add_noise(inputs, sigma)
        noisy_image = (noisy_inputs.squeeze().numpy() * 255).astype(np.uint8)
        if i <= 100:
            new_image = torch.cat((inputs, noisy_inputs), 0)
            save_noisy_image(new_image, f"C:/Users/IICT3/Bithi/Noise regression/Output100/{j}.png")
        i += 1
        save_noisy_image(noisy_inputs, f"C:/Users/IICT3/Bithi/Noise regression/Noisy image/{j}.png")
        j += 1
    sigma_values = np.array(sigma_values)
    return sigma_values

In [10]:
sigma_values = customFunction()

In [15]:
input_dir = "C:/Users/IICT3/Bithi/Noise regression/Noisy image"
dataset = CustomDataset(image_folder=input_dir, sigma_values=sigma_values, transform=transform, device=device)

In [16]:
# Cross-Validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [17]:
def train(model, train_loader, criterion, optimizer, epoch, writer):
    model.train()
    running_loss = 0.0
    predictions = []
    targets = []

    for batch_idx, (inputs, targets_batch) in enumerate(train_loader):
        inputs = inputs.to(device).float()  # Convert inputs to float32
        targets_batch = targets_batch.unsqueeze(1).to(device).float()  # Convert targets to float32
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predictions.extend(outputs.cpu().detach().numpy())
        targets.extend(targets_batch.cpu().detach().numpy())

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(inputs)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    avg_loss = running_loss / len(train_loader)
    print(f'Train Epoch: {epoch}\tAverage Loss: {avg_loss:.6f}')
    mae, rmse, r2 = calculate_metrics(predictions, targets)
    print(f'MAE: {mae:.6f}, RMSE: {rmse:.6f}, R2 Score: {r2:.6f}')
    writer.add_scalar('Training Loss', avg_loss, epoch)
    writer.add_scalar('Training MAE', mae, epoch)
    writer.add_scalar('Training RMSE', rmse, epoch)
    writer.add_scalar('Training R2 Score', r2, epoch)


In [18]:
def test(model, test_loader, criterion, epoch, writer):
    model.eval()
    test_loss = 0.0
    predictions = []
    targets = []

    with torch.no_grad():
        for inputs, targets_batch in test_loader:
            inputs = inputs.to(device).float()  # Convert inputs to float32
            targets_batch = targets_batch.unsqueeze(1).to(device).float()  # Convert targets to float32
            outputs = model(inputs)
            test_loss += criterion(outputs, targets_batch).item()

            predictions.extend(outputs.cpu().detach().numpy())
            targets.extend(targets_batch.cpu().detach().numpy())

    avg_loss = test_loss / len(test_loader)
    print(f'Test Epoch: {epoch}\tAverage Loss: {avg_loss:.6f}')
    mae, rmse, r2 = calculate_metrics(predictions, targets)
    print(f'MAE: {mae:.6f}, RMSE: {rmse:.6f}, R2 Score: {r2:.6f}')
    writer.add_scalar('Testing Loss', avg_loss, epoch)
    writer.add_scalar('Testing MAE', mae, epoch)
    writer.add_scalar('Testing RMSE', rmse, epoch)
    writer.add_scalar('Testing R2 Score', r2, epoch)
    #plot_predictions(predictions, targets, epoch)

In [19]:
def calculate_metrics(predictions, targets):
    predictions = np.array(predictions).reshape(-1)
    targets = np.array(targets).reshape(-1)
    mae = mean_absolute_error(targets, predictions)
    rmse = mean_squared_error(targets, predictions, squared=False)
    r2 = r2_score(targets, predictions)
    return mae, rmse, r2

In [20]:
def plot_predictions(predictions, targets, epoch):
    plt.figure(figsize=(10, 5))
    plt.scatter(targets, predictions, alpha=0.5)
    plt.xlabel('Ground Truth')
    plt.ylabel('Predictions')
    plt.title(f'Predictions vs. Ground Truth (Epoch {epoch})')
    plt.grid(True)
    plt.savefig(f'predictions_vs_ground_truth_epoch_{epoch}.png')
    plt.show()

In [21]:
import glob


In [22]:
import warnings

# Ignore specific warnings
warnings.filterwarnings("ignore", message="The parameter 'pretrained' is deprecated")
warnings.filterwarnings("ignore", message="Arguments other than a weight enum or `None` for 'weights' are deprecated")



In [23]:
#Resnet
from torchsummary import summary
class NoiseNet(nn.Module):
    def __init__(self):
        super(NoiseNet, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Sequential(nn.Linear(self.model.fc.in_features, 512),
                                  nn.ReLU(),
                                  nn.Dropout(0.25),
                                  nn.Linear(512,256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, 1)
)

    def forward(self, x):
        return self.model(x)

model = NoiseNet().to(device)
#summary(model1,(3,227,227))


In [24]:
#ResNet

from tensorboard.backend.event_processing.event_file_loader import EventFileLoader

# Initialize empty lists to store metrics
train_loss_list = []
test_loss_list = []
mae_list = []
rmse_list = []
r2_list = []

# Training and testing loop with cross-validation
epochs = 30
loss_fn = nn.MSELoss()


for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}")
    train_subset = torch.utils.data.Subset(dataset, train_idx)
    test_subset = torch.utils.data.Subset(dataset, test_idx)

    trainloader = DataLoader(train_subset, batch_size=128, shuffle=True)
    testloader = DataLoader(test_subset, batch_size=128, shuffle=False)

    model = NoiseNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    custom_run_name = f"ResNet_fold_{fold + 1}"
    writer = SummaryWriter(log_dir=f"runs/{custom_run_name}")

    for epoch in range(1, epochs + 1):
        train(model, trainloader, loss_fn, optimizer, epoch, writer)
        test(model, testloader, loss_fn, epoch, writer)

    writer.close()

    # Read scalar values from tensorboard event files

    event_file = glob.glob(f"runs/{custom_run_name}/events.out.tfevents.*")[0]
    event_loader = EventFileLoader(event_file)
    scalar_events = [event for event in event_loader.Load()]


    # Extract the required scalar values
    train_loss = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 1), None)
    test_loss = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 2), None)
    mae = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 3), None)
    rmse = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 4), None)
    r2 = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 5), None)


    # Append the metrics to the respective lists
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    mae_list.append(mae)
    rmse_list.append(rmse)
    r2_list.append(r2)



Fold 1
Train Epoch: 1 [0/8000 (0%)]	Loss: 0.030060
Train Epoch: 1	Average Loss: 0.054068
MAE: 0.100787, RMSE: 0.233383, R2 Score: -16.086908
Test Epoch: 1	Average Loss: 0.012813
MAE: 0.062104, RMSE: 0.112315, R2 Score: -3.074363
Train Epoch: 2 [0/8000 (0%)]	Loss: 0.003802
Train Epoch: 2	Average Loss: 0.004502
MAE: 0.055220, RMSE: 0.067111, R2 Score: -0.412891
Test Epoch: 2	Average Loss: 0.003790
MAE: 0.051409, RMSE: 0.061492, R2 Score: -0.221299
Train Epoch: 3 [0/8000 (0%)]	Loss: 0.004180
Train Epoch: 3	Average Loss: 0.004161
MAE: 0.053556, RMSE: 0.064489, R2 Score: -0.304651
Test Epoch: 3	Average Loss: 0.003299
MAE: 0.048885, RMSE: 0.057383, R2 Score: -0.063513
Train Epoch: 4 [0/8000 (0%)]	Loss: 0.004520
Train Epoch: 4	Average Loss: 0.004026
MAE: 0.053131, RMSE: 0.063486, R2 Score: -0.264403
Test Epoch: 4	Average Loss: 0.003376
MAE: 0.049283, RMSE: 0.058029, R2 Score: -0.087611
Train Epoch: 5 [0/8000 (0%)]	Loss: 0.003967
Train Epoch: 5	Average Loss: 0.003893
MAE: 0.052319, RMSE: 0.062

In [53]:
%reload_ext tensorboard

In [63]:
tensorboard --logdir runs